&nbsp;
&nbsp;
&nbsp;

# **IMPORTACIONES**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split     #separación del dataset

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras import regularizers        #weight regularization

from sys import getsizeof                    #Ver tamaño de las variables en Megabytes 
import gc                                    #Liberar memoria

import time

from sklearn.utils import class_weight 

&nbsp;
&nbsp;
&nbsp;

# **VARIABLES**

In [ ]:
SEED = 42                  #Semilla para garantizar la reproducibilidad del programa (asignada a parámetros random_state y seed)

&nbsp;
&nbsp;
&nbsp;

# **FUNCIONES**

In [ ]:
# Carga y Ajuste del dataset de entrenamiento
def load_traindf():
    
    traindf = pd.read_csv('../input/landmark-recognition-2021/train.csv')
    
    #Añadir a train.csv la columna con la dirección de cada imagen para su posterior lectura
    traindf['img_path'] = traindf['id'].apply(lambda r: os.path.join('../input/landmark-recognition-2021/train', r[0], r[1], r[2], r + '.jpg'))
    
    #Conversión de columna landmark_id de int64 a int32 para reducir consumo de memoria
    traindf['landmark_id'] = traindf['landmark_id'].apply(lambda x: np.int32(x))
    
    return traindf


# Lectura y Redimensionamiento imágenes a partir de su ruta (path)
def img_read_resize(img_path): 
    img = plt.imread(img_path)
    img_redim = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
    return img_redim

# Rotación solo ortogonal de imágenes empleada en el generador de imágenes de entrenamiento
def orthogonal_rot(image):
    return np.rot90(image, np.random.choice([-1, 0, 1]))

# Obtener la memoria de cualquier objeto en Bytes. Referencia: https://goshippo.com/blog/measure-real-size-any-python-object/
import sys
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

&nbsp;
&nbsp;
&nbsp;

# **DATASET DE ENTRENAMIENTO**

### Carga del dataset de entrenamiento

In [ ]:
traindf = load_traindf()        #Función previamente definida para cargar el dataset ajustado

landmark_unique = len(traindf['landmark_id'].unique())    #Clases totales del dataset (monumentos diferentes) 
            # traindf['landmark_id'].nunique()
traindf.head()

In [ ]:
traindf.info(memory_usage='deep')

### Información numérica del dataset de entrenamiento

In [ ]:
print('Datos del dataset de entrenamiento \n')
print('Número de imágenes en el dataset a clasificar: ', traindf.shape[0])
print('Número de monumentos (clases) diferentes: ', landmark_unique)
print('Clase más alta: ', max(traindf['landmark_id']))
print('Repeticiones de elementos por clase: Mínimo', min(traindf['landmark_id'].value_counts()),
      'y Máximo',max(traindf['landmark_id'].value_counts()))

In [ ]:
print(traindf['landmark_id'].value_counts())

> Se puede observar que el dataset no está balanceado **(Imbalanced Classification)** por la diferencia entre las frecuencias de las clases. Es necesario realizar oversampling o undersampling. En este programa se realizará **undersampling** en el dataset antes de realizar el split datos entrenamiento-validación, pues es más conveniente disminuir la cantidad de datos de las clases con mayor frecuencia para ahorrar memoria que aumentar los datos de las clases con poca frecuencia dada a la limitación de memoria del sistema. Si esta no fuera un problema, sería recomendable realizar el oversampling, obteniendo así una mayor precisión del modelo.

### Undersampling del dataset de entrenamiento

In [ ]:
UMBRAL_UNDERSAMPLING = 200           #Las clases con número de imágenes > umbral serán reducidas a umbral imágenes aleatorias de esa clase

traindf_unders = traindf.groupby('landmark_id', group_keys=False).apply(lambda x: x.sample(n = min(len(x), UMBRAL_UNDERSAMPLING), random_state= SEED))
traindf_unders.reset_index(inplace=True, drop=True)     #Reiniciar índices. drop=True para que los índices antiguos no sean nueva columna

#Frecuencias de las clases tras undersampling en dataset de entrenamiento
traindf_unders['landmark_id'].value_counts()

In [ ]:
print('Número de imágenes en el dataset tras undersampling: ', traindf_unders.shape[0])

### Contenido del dataset de entrenamiento y generación de la muestra

In [ ]:
'''
#SELECCIONAR MUESTRA SEGÚN NÚMERO DE DATOS (PROBLEMA: una muestra puede acabar en mitad de los datos de una clase; la clase estaría dividida en varias muestras)

IMG_SIZE = 160        #Tamaño/resolución de las imágenes que se empleará en todo el programa
N_DATOS = 10000       #Número de filas del dataframe tras undersampling que se usarán en la muestra

traindf_s = traindf_unders.iloc[:N_DATOS,:]             #Muestra del dataset de entrenamiento tras undersampling

#Si se quieren barajar los datos :
#traindf.sample(frac=N_DATOS/traindf_unders.shape[0], random_state=SEED).reset_index(drop=True)  #Se obtiene un [frac %]  de filas del dataframe original y se barajan

'''

In [ ]:
#SELECCIONAR MUESTRA SEGÚN CLASES MAX Y MIN (TODOS LOS DATOS COMPRENDIDOS ENTRE DICHAS CLASES Y DE DICHAS CLASES)

IMG_SIZE = 220        #Tamaño/resolución de las imágenes que se empleará en todo el programa

CLASE_MIN = 0
CLASE_MAX = 50

traindf_s = traindf_unders[(CLASE_MIN<traindf_unders['landmark_id']) & (traindf_unders['landmark_id']<=CLASE_MAX)]

N_DATOS = len(traindf_s['landmark_id'])

CLASE_MIN = min(traindf_s['landmark_id'])      #Clase mínima real de la muestra
CLASE_MAX = max(traindf_s['landmark_id'])      #Clase máxima real de la muestra ----> En la siguiente muestra, su valor pasaría a CLASE_MIN

print('Número de datos de la muestra, N_DATOS: '+str(N_DATOS))
print('\nClase más baja de la muestra, CLASE_MIN: '+str(CLASE_MIN))
print('\nClase más alta de la muestra, CLASE_MAX: '+str(CLASE_MAX))

In [ ]:
print('ELIMINANDO ' + str(get_size(traindf_unders)/(1024*1024)) +' MB ocupados por traindf_unders')
del traindf_unders         #Elimina la relación entre la variable y la memoria a la que apunta
gc.collect();              #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
traindf_s

In [ ]:
#Comparación de datos en el mismo intervalo de clases [CLASE_MIN, CLASE_MAX]

# Muestra del dataset SIN undersampling
traindf_no_unders = traindf[(CLASE_MIN<traindf['landmark_id']) & (traindf['landmark_id']<=CLASE_MAX)]
plt.figure(1, figsize = (22, 5))
plt.title('Histograma de landmark_id en muestra SIN undersampling: '+ str(len(traindf_no_unders))+' datos', fontweight ="bold")
plt.hist(traindf_no_unders['landmark_id'], color = 'red', bins= CLASE_MAX - CLASE_MIN, histtype= 'stepfilled')
plt.xlabel('landmark_id')
plt.axhline(y=UMBRAL_UNDERSAMPLING, color='g', linestyle='--', label= 'Umbral Undersampling')      #Línea horizontal indicando umbral
plt.legend()
plt.show()

# Muestra del dataset CON undersampling
plt.figure(2, figsize = (22, 5))
plt.title('Histograma de landmark_id en la muestra CON undersampling traindf_s: '+ str(N_DATOS)+' datos', fontweight = "bold")
plt.hist(traindf_s['landmark_id'], color = 'blue', bins = CLASE_MAX - CLASE_MIN, histtype= 'stepfilled')
plt.xlabel('landmark_id')
plt.ylim([0,max(traindf_no_unders['landmark_id'].value_counts())])
plt.axhline(y=UMBRAL_UNDERSAMPLING, color='g', linestyle='--', label= 'Umbral Undersampling')      #Línea horizontal indicando umbral
plt.legend()
plt.show()

> Se ha conseguido equilibrar en cierta medida las frecuencias de las clases a costa de disminuir los datos.

In [ ]:
traindf_s['landmark_id'].value_counts().describe()

In [ ]:
#Eliminar de memoria el dataset completo; solo se usará la muestra tras subsampling durante la ejecución
print('ELIMINANDO ' + str(get_size(traindf)/(1024*1024)) +' MB ocupados por traindf (dataset de entrenamiento completo)')
del traindf          #Elimina la relación entre la variable y el espacio de memoria al que apunta
gc.collect();        #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
print('ELIMINANDO ' + str(get_size(traindf_no_unders)/(1024*1024)) +' MB ocupados por traindf_no_unders (muestra sin undersampling)')
del traindf_no_unders         #Elimina la relación entre la variable y el espacio de memoria al que apunta
gc.collect();                 #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
#Mostrar imágenes aleatorias de la muestra

plt.figure(figsize=(25,7))

for i in range(12):
    j = np.random.randint(0, N_DATOS)              #Cambiar j por i en las indexaciones si se quieren imágenes aleatorias de la muestra
    img = img_read_resize(traindf_s['img_path'][j])     #Para que las imágenes tengan el mismo tamaño (= número de píxeles = neuronas de entrada)
    plt.subplot(2 , 6, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('landmark_id: '+ str(traindf_s['landmark_id'][j]), fontweight ="bold", fontsize=12)
    plt.imshow(img)
    
plt.show()

### Redimensionamiento de las imágenes del dataset de entrenamiento. Almacenamiento de las imágenes en una variable y de las etiquetas en otra

In [ ]:
start_time = time.time()

X = []   #Imágenes
y = []   #Clases
###datos_entrenamiento=[]   #Lista con imágenes y clases

for i in range(traindf_s.shape[0]):                      #Relleno de una lista con imágenes redimensionadas y otra con sus clases  
    X.append(img_read_resize(traindf_s['img_path'][i])) 
    y.append(np.array(traindf_s['landmark_id'][i]))      #Cada etiqueta se convierte en numpy.ndarray (por compatibilidad con las imágenes en X, que son ndarrays)
    ###datos_entrenamiento.append([X[i],y[i]])              #Lista de np.ndarrays  (imágenes y etiquetas)

####df_train = pd.DataFrame(datos_entrenamiento, columns = ['img','landmark_id'])        #Convertimos lista a pd.DataFrame

print('Tipos de las variables: \n')
print('X: ', type(X))
print('Elementos de X: ', type(X[0]))
print('\ny: ', type(y))
print('Elementos de y: ', type(y[0]))

print('\nDuración del redimensionamiento: %s segundos' % (time.time() - start_time))

In [ ]:
###print('ELIMINANDO ' + str(get_size(df_train)/(1024*1024)) +' MB ocupados por df_train')
###del df_train         #Elimina la relación entre la variable y la memoria a la que apunta
###gc.collect();        #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
print('ELIMINANDO ' + str(get_size(traindf_s)/(1024*1024)) +' MB ocupados por traindf_s')
del traindf_s         #Elimina la relación entre la variable y la memoria a la que apunta
gc.collect();         #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
whos

### Ajustes a las variables

In [ ]:
#Conversión de lista X a array + Normalización de las imágenes en X + Cambio de tipo de int8 a float16 para reducir memoria
X = np.array(X).astype('float16')     #Se normalizan las imágenes para que los valores de los píxeles estén entre [0,1] en vez de [0,255]

#Conversión de lista y a array
y = np.array(y)

### Obtención de los datos de validación a partir del dataset de entrenamiento

In [ ]:
#Separación de los datos en datos para entrenamiento y datos para validación

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.15, random_state=SEED, shuffle=True)  #Mismo tipo que X e y (ndarray formado por ndarrays o int64, respectivamente)

print('Imágenes en X: ',len(X), ' y etiquetas en y: ', len(y))
print('Entrenamiento. Imágenes en X_train: ',len(X_train), 'y etiquetas en y_train: ',len(y_train))
print('Validación. Imágenes en X_val: ',len(X_val), 'y etiquetas en y_val: ',len(y_val))

In [ ]:
print('ELIMINANDO ' + str(get_size(X)/(1024*1024)) +' MB ocupados por X')
del X         #Elimina la relación entre la variable y la memoria a la que apunta
gc.collect();        #Elimina de memoria y devuelve los objetos a los que ya no se apunta

In [ ]:
print('ELIMINANDO ' + str(get_size(y)/(1024*1024)) +' MB ocupados por y')
del y         #Elimina la relación entre la variable y la memoria a la que apunta
gc.collect();       #Elimina de memoria y devuelve los objetos a los que ya no se apunta

### Aumento de datos

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 30,            #Grados
    #preprocessing_function = orthogonal_rot,          #----> Ralentiza muchísimo el entrenamiento
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    #shear_range = 5,               #Grados
    #zoom_range = 0.3,              #----> Ralentiza mucho el entrenamiento
    fill_mode = 'nearest',
    #horizontal_flip = True,
    #vertical_flip = False,   
)
#train_datagen.fit(X_train)

val_datagen = ImageDataGenerator(
    rescale=1./255,
)

#Mostrar ejemplos de imágenes transformadas por el generador de entrenamiento
plt.figure(figsize=(25,7))
for img, clase in train_datagen.flow(X_train, y_train, batch_size = 12, seed=SEED, shuffle=True):
    for i in range(12):
        plt.subplot(2 , 6, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel('landmark_id: '+ str(clase[i]), fontweight ="bold", fontsize=12)
        plt.imshow(img[i])
    plt.show()
    break;

&nbsp;
&nbsp;
&nbsp;

# **HIPERPARÁMETROS DEL MODELO**

In [ ]:
#Parámetros de la red neuronal convolucional

kernelSize = (3,3)            #Tamaño de la plantilla de convolución
paddingType = 'same'          #Cómo se procede en los bordes de las imágenes ('same' o 'valid')
activationF = 'relu'          #Función de activación
poolSize = (2,2)              #Tamaño de la plantilla de maximum pooling
stridesSize = (2,2)           #Desplazamiento de plantilla durante maximum pooling
dropoutRate = 0.5             #Porcentaje de neuronas que se desactivan con la capa Dropout
#batchSize (tomará valores en un bucle)   #Cantidad de datos con los que se entrena en cada época (tomará valores en un bucle)
epochsSize= 500               #Número de épocas en las que se entrena
lr = 0.01                     #Learning Rate

weightDecay = regularizers.L2(0.01) #Regularización de pesos ---> L1: Suma pesos absolutos. L2: Suma pesos cuadrados. L1L2: Suma pesos absolutos y cuadrados

## En caso de utilizar learning_rate personalizado

LR_START = 0.001
LR_MAX = 0.005
LR_MIN = 0.001
LR_RAMPUP_EPOCHS = 75
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .9

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(epochsSize)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

&nbsp;
&nbsp;
&nbsp;

# **FUNCIÓN PARA CREACIÓN, COMPILACIÓN Y ENTRENAMIENTO DEL MODELO**

In [ ]:
def crear_modelo(batchSize):
    
    #Creación del modelo

    model = keras.Sequential([
      
        #BASE DEL MODELO ---- Extracción de características

        #Bloque convolucional 1  
        layers.Conv2D(filters=32, kernel_size=kernelSize, strides=1, padding=paddingType, activation=activationF, input_shape=[IMG_SIZE, IMG_SIZE, 3]),
        layers.BatchNormalization(),
        layers.MaxPool2D(pool_size=poolSize, strides=stridesSize, padding=paddingType),

        #Bloque convolucional 2  
        layers.Conv2D(filters=64, kernel_size=kernelSize, strides=1, padding=paddingType, activation=activationF),
        layers.BatchNormalization(),
        layers.MaxPool2D(pool_size=poolSize, strides=stridesSize, padding=paddingType),

        #Bloque convolucional 3  
        layers.Conv2D(filters=128, kernel_size=kernelSize, strides=1, padding=paddingType, activation=activationF),
        layers.BatchNormalization(),
        layers.MaxPool2D(pool_size=poolSize, strides=stridesSize, padding=paddingType),

        #Bloque convolucional 4  
        layers.Conv2D(filters=256, kernel_size=kernelSize, strides=1, padding=paddingType, activation=activationF),
        layers.BatchNormalization(),
        layers.MaxPool2D(pool_size=poolSize, strides=stridesSize, padding=paddingType),
        
        #Bloque convolucional 5 
        #layers.Conv2D(filters=512, kernel_size=kernelSize, strides=1, padding=paddingType, activation=activationF),
        #layers.BatchNormalization(),
        #layers.MaxPool2D(pool_size=poolSize, strides=stridesSize, padding=paddingType),
        

        #CABEZA DEL MODELO ---- Clasificación

        layers.Flatten(),
        layers.BatchNormalization(),
        layers.Dropout(rate=dropoutRate),
        layers.Dense(units = 512, activation=activationF, kernel_regularizer=weightDecay), #Para 1 hidden layer, n_neuronas = sqrt(n_input*n_output) + [1,10]

        layers.BatchNormalization(),
        layers.Dropout(rate=dropoutRate),
        layers.Dense(units = landmark_unique, activation='softmax'),         #Número de neuronas = Número de clases TOTALES     
    ])
    
    #OPTIMIZADORES
    # A menor lr, menos cambios repentinos hay en las métricas como acuracy
    opt1 = tf.keras.optimizers.RMSprop(learning_rate=lr, rho=0.9, momentum=0.5, epsilon=1e-07)   
    opt2 = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
    opt3 = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=False)         #momentum--> acelera gradiente y amortigua oscilaciones
    
    #Compilación del modelo   
    model.compile(
        optimizer = opt3,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['sparse_categorical_accuracy']
    )
    
    #Callback 1
    early_stopping = EarlyStopping(
        monitor = "val_loss",
        mode = "auto",
        min_delta = 0.0001,         # minimium amount of change to count as an improvement
        patience = 30,              # how many epochs to wait before stopping
        restore_best_weights = True,
    )
    
    #Callback 2
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=7, cooldown=1, min_lr=0.0005, min_delta=0.001, verbose=1)
    
    
    #Entrenamiento del modelo
    history = model.fit(
        train_datagen.flow(X_train, y_train, batch_size = batchSize),             #Entrena con X_train datos en cada época, empleando en cada step batch_size datos
                                                                                  # de X_train modificados aleatoriamente según se indicó en el generador
        validation_data = val_datagen.flow(X_val, y_val, batch_size = batchSize),
        class_weight = dic_class_weights,      #Para solucionar Imbalanced Data
        #shuffle = True,                       #Al trabajar con un generador de datos se ignora
        batch_size = None,                     #batch_size indicado en generador(train_datagen)
        steps_per_epoch = len(X_train)//batchSize,
        epochs = epochsSize,
        callbacks = [early_stopping, reduce_lr],
        verbose=1,            # 0: silencio     1: barra de progreso + texto      2: solo texto
        #use_multiprocessing=True,
        #max_queue_size = 15,        #Default = 10
        #workers = 32,
    ) 
  
    return model, history

&nbsp;
&nbsp;
&nbsp;

# **GRÁFICAS DE PÉRDIDA Y PRECISIÓN Y EVALUACIÓN DEL MODELO**

In [ ]:
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
'''

In [ ]:
# Cálculo de pesos de cada clase de los datos para entrenamiento de la muestra
_, freq = np.unique(y_train, return_counts=True)
max_freq = np.max(freq)
print ('Mayor frecuencia: '+str(np.max(freq)))

##Método no funciona correctamente con este set debido a la numeración de landmark_id
##classWeights = class_weight.compute_class_weight(class_weight ='balanced',classes = np.unique(y_train), y = y_train)
##train_classWeights = dict(enumerate(classWeights))

#Se realiza este procedimiento porque con el método compute_class_weight no funciona --> Las keys del diccionario deben coincidir con las etiquetas
dic_class_weights ={}
i=0
for i in range(max(y_train)+1):                   #Se va a crear un diccionario de max(y_train)+1 elementos, aunque el número de clases sea menor
    dic_class_weights[i] = 0                      #Inicialmente todos los pesos (valores del diccionario) a 0
    if i in y_train:                              #Si la clave coincide con el landmark_id (etiqueta) el peso no será 0
        freq = len(y_train[y_train == i])         #Frecuencia de la etiqueta i
        dic_class_weights[i] = max_freq/freq

In [ ]:
batchSize = [64]
#[2, 4, 8, 16, 32, 64, 128]

min_val_loss = []
max_val_acc = []

start_time = time.time()

for j in range(len(batchSize)):           #En cada iteración se crea el modelo DESDE CERO con la función crear_modelo()

    model, history = crear_modelo(batchSize[j])
    
    history_df = pd.DataFrame(history.history)    
    
    print('\nbatchSize = '+str(batchSize[j]))
    
    plt.figure(figsize=(15,5))        #Anchura y Altura de las gráficas, respectivamente
    
    plt.subplot(1,2,1)
    #history_df.loc[0:, ['loss', 'val_loss']].plot()             #Utilizar dataframe y su método plot dan problemas con plt.subplot
    plt.plot(history.history['loss'], label='train')        
    plt.plot(history.history['val_loss'], label='test')
    plt.title('Loss and Validation Loss')
    plt.xlabel('batchSize = '+str(batchSize[j]))
    print(("Minimum Validation Loss: {:0.4f} in epoch {:0.0f} ").format(history_df['val_loss'].min(), history_df['val_loss'].idxmin()))            
    
    plt.subplot(1,2,2)
    #history_df.loc[0:, ['accuracy', 'val_accuracy']].plot()
    plt.plot(history.history['sparse_categorical_accuracy'], label='train')
    plt.plot(history.history['val_sparse_categorical_accuracy'], label='test')
    plt.title('Accuracy and Validation Accuracy')
    plt.xlabel('batchSize = '+str(batchSize[j]))
    print(("Maximum Validation Accuracy: {:0.4f} in epoch {:0.0f} ").format(history_df['val_sparse_categorical_accuracy'].max(), history_df['val_sparse_categorical_accuracy'].idxmax()))

    print("\nEvaluación del modelo con datos de entrenamiento")
    score = model.evaluate(X_train, y_train)
    print("Test loss, Test accuracy:", score[0], score[1])

    print("\nEvaluación del modelo con datos de validación")
    score = model.evaluate(X_val, y_val)
    print("Test loss, Test accuracy:", score[0], score[1])

    plt.show()    #Se muestran las gráficas
    
    #Proceso para almacenar los mejores resultados (menor pérdida y mayor precisión)
    if j==0:
        min_val_loss= [history_df['val_loss'].min(), history_df['val_loss'].idxmin(), batchSize[j]]
    else:
        if min_val_loss[0]> history_df['val_loss'].min():
            min_val_loss= [history_df['val_loss'].min(), history_df['val_loss'].idxmin(), batchSize[j]]
            
    if j==0:
        max_val_acc= [history_df['val_sparse_categorical_accuracy'].max(), history_df['val_sparse_categorical_accuracy'].idxmax(), batchSize[j]]
    else:
        if max_val_acc[0]< history_df['val_sparse_categorical_accuracy'].max():
            max_val_acc= [history_df['val_sparse_categorical_accuracy'].max(), history_df['val_sparse_categorical_accuracy'].idxmax(), batchSize[j]]
            
            
print('\nBest Results:')            
print(("Minimum Validation Loss: {:0.4f} in epoch {:0.0f} with batchSize = {:0.0f}").format(min_val_loss[0], min_val_loss[1], min_val_loss[2]))
print(("Maximum Validation Accuracy: {:0.4f} in epoch {:0.0f} with batchSize = {:0.0f}").format(max_val_acc[0], max_val_acc[1], max_val_acc[2]))
print('\nDuración del entrenamiento: %s minutos' % ((time.time() - start_time)/60))

In [ ]:
model.summary()

In [ ]:
#Guardado del modelo completo

model.save('./MyModel_h5',save_format='h5')  
#model.save('./MyModel_tf',save_format='tf')

#Guardado de pesos del modelo
model.save_weights('./Model_Weights_h5', save_format='h5')

In [ ]:
#Cargado de un modelo guardado

model_cargado = tf.keras.models.load_model('./MyModel_h5')

In [ ]:
'''
#En caso de que sea necesario borrar algún archivo en el directorio de salida

filename = ''
import os
os.remove(filename)
'''

In [ ]:
'''
#En caso de que sea necesario borrar algún directorio en el directorio de salida
directory = ''
import shutil
shutil.rmtree(directory)
'''

In [ ]:
#model_cargado.get_weights()

#Reentrenar el modelo

model_cargado.fit(
        X_train, y_train,
        validation_data= (X_val, y_val),
        shuffle = True,
        batch_size=batchSize,
        epochs=epochsSize,
        callbacks=[early_stopping],
        verbose=1,    # 0: silencio     1: barra de progreso + texto      2: solo texto)
)

&nbsp;
&nbsp;
&nbsp;

# **PREDICCIÓN DE LOS DATOS DE TESTEO**

### Carga del dataset de testeo y representación de su contenido

In [ ]:
mainpath_test = '../input/landmark-recognition-2021/test'

sample_subm = pd.read_csv('../input/landmark-recognition-2021/sample_submission.csv')         #Leer archivo .csv con imágenes a clasificar

plt.figure(figsize=(25,7))

for i in range(12):
    img_id = sample_subm['id'][i]
    img_path = os.path.join(mainpath_test, img_id[0], img_id[1], img_id[2], img_id + '.jpg')
    img = img_read_resize(img_path)  #Se usa la función previamente definida para lograr el mismo tamaño que las imágenes de entrenamiento
    plt.subplot(2 , 6, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel('Clase: por determinar')
    plt.imshow(img)

### Ajuste de datos y predicción

In [ ]:
X_pred = []
X_pred_copia = []
confianza = []

#Se rellena una lista con las imágenes de testeo redimensionadas que se vayan a predecir

elem_a_predecir = 30

for i in range(elem_a_predecir): 
    img_id = sample_subm['id'][i]
    img_path = os.path.join(mainpath_test, img_id[0], img_id[1], img_id[2], img_id + '.jpg')
    img = img_read_resize(img_path)               #Se usa la función previamente definida para lograr el mismo tamaño que las imágenes de entrenamiento
    X_pred.append(img)
    X_pred_copia.append(img)                            #Copia de las imágenes de testeo redimensionadas y sin normalizar

X_pred = np.array(X_pred).astype('float16')/255         #Normalización de las imágenes de testeo redimensionadas

predic = model.predict(X_pred, verbose=1)       # batch_size=None,        #Se realiza la predicción

In [ ]:
predic

In [ ]:
predic.shape         #Array de elem_a_predecir filas, neuronas de salida (81313) columnas

In [ ]:
predic[0]        #Primer elemento predicho

In [ ]:
sample_subm.head()      #Archivo .csv antes de introducir resultados

In [ ]:
y_pred = []

for i in range(len(predic)):
    y_pred.append(np.argmax(predic[i]))                                        #Clase (landmark_id) con mayor probabilidad
    confianza.append(predic[i][y_pred[i]].round(2))                            #Probabilidad de la clase con mayor probabilidad
    sample_subm['landmarks'][i] = str(y_pred[i]) +' '+ str(confianza[i])       #Se añaden los resultados al archivo .csv

In [ ]:
sample_subm.head()         #Archivo .csv después de introducir los resultados

### Representación visual de los resultados

In [ ]:
#Leer de nuevo el dataset de entrenamiento pues se eliminó de la memoria
traindf = load_traindf()

In [ ]:
col = 5       #Número de columnas del subplot (teniendo en cuenta la propia imagen a predecir)

k=0
for k in range(elem_a_predecir):
    plt.figure(figsize=(16,7))
    img_pred = X_pred_copia[k]      #Cualquier imagen que se predijo anteriormente (ya está redimensionada)
    plt.subplot(1, col, 1)
    plt.xticks([])
    plt.yticks([])
    plt.title('Imagen a predecir nº '+ str(k))
    str1 = 'Clase predicha: ' + str(y_pred[k]) 
    str2 = 'Confianza: ' + str(confianza[k])
    plt.xlabel(str1 + '\n' + str2, fontsize = 12, weight = 'bold')             #Para imprimir en 2 líneas distintas
    plt.imshow(img_pred)
    
    i=0
    img_clase_df = traindf[traindf['landmark_id']==y_pred[k]]            #Dataframe con imágenes de igual 'landmark_id' que la predicción
    
    for i in range(len(img_clase_df)):
        if i < (col-1): 
            img_clase_path = img_clase_df.iloc[i,2] 
            img_clase = img_read_resize(img_clase_path)
            plt.subplot(1, col, i+2)
            plt.xticks([])
            plt.yticks([])
            plt.title('Imagen de la clase '+ str(y_pred[k]))
            plt.xlabel(str())
            plt.imshow(img_clase)
        else:
            break;